In [4]:
import string
import sys
import re
from pickle import dump
import unicodedata
from unicodedata import normalize
import numpy

import indicnlp
INDIC_NLP_LIB_HOME=r"/home/nitin_grover/Desktop/Neural Machine Translation/English_Hindi/indic_nlp_library"
INDIC_NLP_RESOURCES=r"/home/nitin_grover/Desktop/Neural Machine Translation/English_Hindi/indic_nlp_resources-master"
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))


In [6]:
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader


In [9]:
#loader.load()
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize

In [15]:

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

#load doc into memory
def load_doc(filename):
    file = open(filename,mode='rt',encoding='utf-8')
    text = file.read()
    file.close()
    return text

#function to clean the text
def clean_text(text):
    text = text.replace(u',','')
    text = text.replace(u'"','')
    text = text.replace(u'"','')
    text = text.replace(u"‘‘",'')
    text = text.replace(u"’’",'')
    text = text.replace(u"''",'')
    text = text.replace(u"।",'')
    text=text.replace(u',','')
    text=text.replace(u'"','')
    text=text.replace(u'(','')
    text=text.replace(u')','')
    text=text.replace(u'"','')
    text=text.replace(u':','')
    text=text.replace(u"'",'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text=text.replace("'", "")
    text=text.replace('"', "")
    text= re.sub("'", '', text)
    text= re.sub("’", '', text)
    text=re.sub('[0-9+\-*/.%]', '', text)
    text=text.strip()
    text=re.sub(' +', ' ',text)
    exclude = set(string.punctuation)
    text= ''.join(ch for ch in text if ch not in exclude)
    return text

#functions to preprocess English and Hindi text
def pre_process_english_sentence(line):
    line = line.lower()
    line = clean_text(line)
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    line = normalize('NFD', line).encode('ascii', 'ignore')
    line = line.decode('UTF-8')
    line = line.split()
    line = [re_print.sub('', w) for w in line]
    line = [word for word in line if word.isalpha()]
    line = ' '.join(line)
    return line

def pre_process_hindi_sentence(line):
    line=re.sub('[a-zA-Z]', '', line)
    line = clean_text(line)
    remove_nuktas = False
    factory = IndicNormalizerFactory()
    normalizer = factory.get_normalizer("hi")
    line = normalizer.normalize(line)
    tokens = list()
    for t in indic_tokenize.trivial_tokenize(line):
        tokens.append(t)
    line = tokens
    line = [word for word in line if not re.search(r'\d', word)]
    line = ' '.join(line)
    return (line)


# split the loaded document into sentences and combine them to 
# form pairs
def to_pairs(eng_doc,hindi_doc):
    eng_lines = eng_doc.strip().split('\n')
    hindi_lines = hindi_doc.strip().split('\n')
    pairs = []
    for line in range(len(eng_lines)):
        pairs.append([])
        pairs[line].append(pre_process_english_sentence(eng_lines[line]))
        pairs[line].append(pre_process_hindi_sentence(hindi_lines[line]))
    return pairs

def prepareData(pairs):
    input_lang = Lang('eng')
    output_lang = Lang('hindi')
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    return input_lang, output_lang


#save the cleaned data to a file
def save_cleaned_data(sentences,filename):
    dump(sentences,open(filename,'wb'))
    print('Saved: %s' %filename)

In [16]:
english_text = load_doc('/home/nitin_grover/Desktop/Neural Machine Translation/English_Hindi/train/alt/en-hi/train.en')
hindi_text = load_doc('/home/nitin_grover/Desktop/Neural Machine Translation/English_Hindi/train/alt/en-hi/train.hi')

pairs_all = to_pairs(english_text, hindi_text)

input_lang, output_lang = prepareData(pairs_all)

In [14]:
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
input_text=u"\u0929 \u0928\u093c"
remove_nuktas=False
factory=IndicNormalizerFactory()
normalizer=factory.get_normalizer("hi")
print (normalizer.normalize(input_text))

ऩ ऩ


In [20]:
pairs_all = numpy.array(pairs_all)
pairs_all.shape

(20106, 2)

In [21]:
pairs_all

array([['italy have defeated portugal in pool c of the rugby world cup at parc des princes paris france',
        'में फ़्रांस पेरिस के पार्क डेस प्रिंसेस में हुए रग्बी विश्व कप के पूल में इटली ने पुर्तगाल को से हराया'],
       ['andrea masi opened the scoring in the fourth minute with a try for italy',
        'एंड्रिया मैसी ने चौथे मिनट में इटली के लिए ट्राई के द्वारा स्कोरिंग की शुरुआत करी'],
       ['despite controlling the game for much of the first half italy could not score any other tries before the interval but david bortolussi kicked three penalties to extend their lead',
        'पहले हाफ़ के अधिकांश समय तक खेल नियंत्रण में रहने के बावजूद इटली अंतराल से पहले कोई अन्य स्कोर नहीं कर पाया लेकिन डेविड बोर्तोलुसी ने अपनी बढ़त को बढ़ाने के लिए तीन पेनाल्टी लगाई'],
       ...,
       ['tmzcom reports that police were checking to see if her license was valid and was surprised that it was',
        'की रिपोर्ट है कि पुलिस यह देखने के लिए जाँच कर रही थी कि उनका लाइसेंस वैध था या नहीं 

In [22]:
input_lang

In [23]:
for i in range(100):
    print('[%s] ==> [%s]' %(pairs_all[i][0],pairs_all[i][1]))

[italy have defeated portugal in pool c of the rugby world cup at parc des princes paris france] ==> [में फ़्रांस पेरिस के पार्क डेस प्रिंसेस में हुए रग्बी विश्व कप के पूल में इटली ने पुर्तगाल को से हराया]
[andrea masi opened the scoring in the fourth minute with a try for italy] ==> [एंड्रिया मैसी ने चौथे मिनट में इटली के लिए ट्राई के द्वारा स्कोरिंग की शुरुआत करी]
[despite controlling the game for much of the first half italy could not score any other tries before the interval but david bortolussi kicked three penalties to extend their lead] ==> [पहले हाफ़ के अधिकांश समय तक खेल नियंत्रण में रहने के बावजूद इटली अंतराल से पहले कोई अन्य स्कोर नहीं कर पाया लेकिन डेविड बोर्तोलुसी ने अपनी बढ़त को बढ़ाने के लिए तीन पेनाल्टी लगाई]
[portugal never gave up and david penalva scored a try in the rd minute providing their only points of the match] ==> [हार न मानते हुए पुर्तगाल के डेविड पेनलावा ने मिनट में पॉइंट स्कोर किया पूरे खेल में यह पुर्तगाल का इकलौता पॉइंट था]
[italy led at half time but we

In [24]:
#save it into a file
save_cleaned_data(pairs_all,'english-hindi.pkl')

Saved: english-hindi.pkl


In [33]:
#Split data into train(18k) and test(2k)
from pickle import load,dump
from numpy.random import rand,shuffle

#load the clean dataset
def load_clean_dataset(filename):
    return load(open(filename,'rb'))

#for saving a list of clean sentences to file
def save_clean_data(sentences,filename):
    dump(sentences,open(filename,'wb'))
    print('Saved: %s' %filename)

raw_dataset = load_clean_dataset('english-hindi.pkl')

#reducing dataset size to 20k
n_sentences = 20000
dataset = raw_dataset[:n_sentences,:]
shuffle(dataset)

train,test = dataset[:18000],dataset[18000:]

#save it 
save_clean_data(dataset,'english-hindi-both.pkl')
save_clean_data(train,'english-hindi-train.pkl')
save_clean_data(test,'english-hindi-test.pkl')

Saved: english-hindi-both.pkl
Saved: english-hindi-train.pkl
Saved: english-hindi-test.pkl


In [34]:
train

array([['in line with a press statement made by the ministry of foreign affairs and european integration mfaei on wednesday december romanian ambassador extraordinary and plenipotentiary to moldova filip teodorescu was invited to the mfaei',
        'विदेश मंत्रालय और यूरोपीय एकीकरण मंत्रालय एमएफएईआई द्वारा दिए गए एक प्रेस बयान के अनुसार बुधवार दिसंबर को मोल्दोवा के लिए रोमानियाई राजदूत असाधारण और साधिकार फिलिप टिओडोरेस्कू एमएफएईआई को आमंत्रित किया गया था'],
       ['fredricks death comes in the same month as gamini goonasena another former sri lankan cricketer who died at the age of',
        'फ्रेड्रिक की मृत्यु उसी महीने में हुई जब गामिनी गोनासेन श्रीलंका के एक अन्य पूर्व क्रिकेटर जिनकी वर्ष की आयु में मृत्यु हो गई थी'],
       ['that cannot will not happen if scotland leaves the union',
        'अगर स्कॉटलैंड संघ को छोड़ता है तो ऐसा नहीं होगा'],
       ...,
       ['leslie aun a spokesperson for the komen foundation said the organization adopted a new rule that does not allow grant

In [35]:
train.shape

(18000, 2)

In [36]:
test.shape

(2000, 2)